# Entregable 2: Construyendo un modelo de clasificación aplicando *fine-tuning*

En este notebook se muestra cómo crear un modelo de clasificación de imágenes utilizando la técnica de *transfer-learning* conocida como *fine-tuning*.

Para ello vamos a utilizar la librería [fastAI](https://www.fast.ai/). Este notebook está inspirado en el curso asociado a dicha librería. 

En esta práctica vamos a hacer un uso intensivo de la GPU, así que es importante activar su uso desde la opción *Notebook settings* del menú *Edit* . 

## Carga de librerías

Comenzamos instalando y cargando las librerías que vamos a necesitar en esta práctica. La librería ``fastai`` nos proporciona los distintos algoritmos de aprendizaje profundo y la librería ``os`` la utilizamos para la gestión de ficheros. 

In [ ]:
!pip install fastai --upgrade

In [ ]:
from fastai.vision.all import *
import os
from google.colab import files
from sklearn.model_selection import train_test_split
import shutil

## Creando nuestro dataset de imágenes

El primer paso en cualquier proyecto de clasificación de imágenes es construir un dataset de imágenes anotadas. Aunque existen muchos datasets disponibles vamos a ver que con las técnicas explicadas en este notebook se pueden construir buenos modelos de clasificación para cualquier problema. 

En concreto, nuestro objetivo va a ser construir un modelo capaz de distinguir entre los personajes de la serie [Futurama](https://es.wikipedia.org/wiki/Futurama). En concreto queremos saber si quien aparece en una imagen es Fry, Bender o Leela. 

<img src="https://purepng.com/public/uploads/large/purepng.com-futurama-leela-fry-benderfuturamaanimationsciencefictioncartoonroberto-17015286039035k1si.png" alt="Fry, Bender y Leela" style="width: 50px;"/>



### Creando el dataset

Para crear nuestro dataset vamos a utilizar imágenes adquiridas desde Google Imágenes. El proceso que seguimos para construir nuestro dataset viene explicado en el blog de [Adrian Rosebrock](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/). En concreto hemos buscado imágenes de Fry, Leela y Bender y hemos creado los ficheros ``fry.csv``,  ``bender.csv`` y  ``leela.csv`` que contienen las URLs donde se encuentran las imágenes de cada uno de estos personajes. 

In [ ]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica3/bender.csv -O bender.csv
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica3/fry.csv -O fry.csv
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica3/leela.csv -O leela.csv

### Descargando las imágenes y organizando las carpetas

Habitualmente para crear modelos de clasificación de imágenes se crea una carpeta para cada clase de imagen (en este caso necesitaremos crear tres carpetas, una para fry, otra para bender y otra para leela).

En nuestro caso además de crear las carpetas debemos descargar las imágenes para ello debemos ejecutar las siguientes celdas (notad que el código es identico en los tres casos y lo único que cambia es la clase de las imágenes).

Es posible que al intentar descargar alguna imagen se produzcan errores, pero no son relevantes (solo indican que esa imagen no se ha podido descargar).



In [ ]:
path = Path('data/futurama')

In [ ]:
folder = 'leela'
file = 'leela.csv'
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
# Como mucho descargamos 200 imágenes
download_images(dest,path/file,max_pics=200)

In [ ]:
folder = 'fry'
file = 'fry.csv'
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
# Como mucho descargamos 200 imágenes
download_images(dest,path/file,max_pics=200)

In [ ]:
folder = 'bender'
file = 'bender.csv'
dest = path/folder
dest.mkdir(parents=True,exist_ok=True)
os.rename(file,path/file)
download_images(dest,path/file,max_pics=200)

Puede ocurrir que algunas imágenes tengan un formato que no pueda ser abierto por la librería, por lo que vamos a eliminarlas con la siguiente instrucción.

In [ ]:
for im in verify_images(get_image_files(path)):
    os.remove(str(im))

A continuación vamos a partir nuestro dataset en un conjunto de entrenamiento y en otro de test (usaremos el 80% de las imágenes para entrenar y el 20% para test). Para partir el dataset debemos organizarlo con la siguiente estructura de directorios:

```
data
└── futurama
    ├── test
    │   ├── bender
    │   ├── fry
    │   └── leela
    └── train
        ├── bender
        ├── fry
        └── leela
```

Es decir tenemos una carpeta train y otra carpeta test. Dentro de la carpeta train tendremos tantas carpetas como clases tiene nuestro dataset, y análogamente para la carpeta test. 

Comenzamos creando la estructura de carpetas.


In [ ]:
(path/'train/bender').mkdir(parents=True,exist_ok=True)
(path/'train/leela').mkdir(parents=True,exist_ok=True)
(path/'train/fry').mkdir(parents=True,exist_ok=True)
(path/'test/bender').mkdir(parents=True,exist_ok=True)
(path/'test/leela').mkdir(parents=True,exist_ok=True)
(path/'test/fry').mkdir(parents=True,exist_ok=True)

A continuación vamos a partir las imágenes descargadas previamente y las almacenamos en la carpeta correspondiente.

In [ ]:
trainBender, testBender = train_test_split(get_image_files(path/'bender'),test_size=0.2,random_state=15)
trainLeela, testLeela = train_test_split(get_image_files(path/'leela'),test_size=0.2,random_state=15)
trainFry, testFry = train_test_split(get_image_files(path/'fry'),test_size=0.2,random_state=15)

In [ ]:
for x in trainBender:
  shutil.move(str(x),path/('train/bender/'+x.name))

for x in trainLeela:
  shutil.move(str(x),path/('train/leela/'+x.name))

for x in trainFry:
  shutil.move(str(x),path/('train/fry/'+x.name))

for x in testBender:
  shutil.move(str(x),path/('test/bender/'+x.name))

for x in testLeela:
  shutil.move(str(x),path/('test/leela/'+x.name))

for x in testFry:
  shutil.move(str(x),path/('test/fry/'+x.name))

Por último eliminamos aquellas carpetas y ficheros que ya no vamos a utilizar. 

In [ ]:
shutil.rmtree('data/futurama/bender')
shutil.rmtree('data/futurama/fry')
shutil.rmtree('data/futurama/leela')
os.remove('data/futurama/bender.csv')
os.remove('data/futurama/fry.csv')
os.remove('data/futurama/leela.csv')

### Cargando el dataset

A continuación vamos a mostrar cómo se carga el dataset para poder posteriormente crear nuestro modelo. Este proceso se hace en dos pasos. Primero se construye un objeto `DataBlock` y a continuación se construye un objeto `DataLoader` a partir del `DataBlock`. Tienes más información sobre estos objetos en la documentación de [FastAI](https://docs.fast.ai/tutorial.datablock.html).

### Datablock

Comenzamos construyendo el objeto `DataBlock`. A continuación explicaremos cada una de sus componentes. 

In [ ]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(valid_pct=0.2,seed=42),
                 get_y=parent_label,
                 item_tfms = Resize(512),
                 batch_tfms=aug_transforms(size=256,min_scale=0.75))

Vamos a ver las distintas componentes del `DataBlock`.

- El atributo `blocks` sirve para indicar el tipo de nuestros datos. Como estamos en un problema de clasificación de imágenes, tenemos que la entrada de nuestro modelo será una imagen, es decir un `ImageBlock`, y la salida será una categoría, es decir un `CategoryBlock`. Por lo tanto indicamos que `blocks = (ImageBlock, CategoryBlock)`.
- El atributo `get_items` debe proporcionar una función para leer los datos. En nuestro caso queremos leer una serie de imágenes que estarán almacenadas en un `path`. Para ello usamos la función `get_image_files`. Puedes ver qué hace exactamente esta función ejecutando el comando `??get_image_files`.
- El atributo `splitter` nos indica cómo partir el dataset. Daros cuenta que tenemos un conjunto de entrenamiento y uno de test, pero para entrenar nuestro modelo y probar distintas alternativas nos interesa usar un conjunto de validación, que lo vamos a tomar de forma aleatorea a partir de nuestro conjunto de entrenamiento usando un 20% del mismo. Para ello usaremos el objeto `RandomSplitter(valid_pct=0.2,seed=42)`.
- El atributo `get_y` sirve para indicar cómo extraemos la clase a partir de nuestros datos. La función `get_image_files` nos proporciona una lista con los paths a las imágenes de nuestro dataset. Si nos fijamos en dichos paths, la clase de cada imagen viene dada por la carpeta en la que está contenida, por lo que podemos usar el método `parent_label` para obtener la clase de la misma. 

Por último, los atributos `item_tfms` y `batch_tfms` sirven para aplicar una técnica conocida como *preescalado* (o *presizing*).   

### Dataloader

Pasamos ahora a construir nuestro `DataLoader` que se construye a partir del `DataBlock` construido anteriormente indicándole el path donde se encuentran nuestras imágenes. Además podemos configurar el `DataLoader` indicándole el tamaño del batch que queremos utilizar. Al trabajar con GPUs es importante que usemos batches de tamaño 2^n para optimizar el uso de la GPU.

In [ ]:
trainPath = Path('data/futurama/train')

In [ ]:
dls = db.dataloaders(trainPath,bs=128)

A continuación mostramos un batch de nuestro `DataLoader`. Es conveniente comprobar que realmente se han cargado las imágenes y sus anotaciones de manera correcta. 

In [ ]:
dls.show_batch()

## Creando el modelo de predicción

A continuación vamos a crear nuestra red convolucional usando *transfer learning* y utilizando como base la arquitectura [ResNet 34](https://arxiv.org/abs/1512.03385); aunque existen otros [modelos disponibles](https://pytorch.org/docs/stable/torchvision/models.html) este modelo proporciona buenos resultados. Al crear nuestra red convolucional también debemos indicar la [métrica](https://docs.fast.ai/metrics.html#metrics) que vamos a utilizar para medir el rendimiento del modelo, en este caso vamos a usar el error_rate y la accuracy.

La primera vez que se ejecuta la siguiente instrucción puede llevar algún tiempo debido a que se tienen que descargar los pesos asociados a la red ResNet 34. 



In [ ]:
learn = cnn_learner(dls,resnet18,metrics=accuracy).to_fp16()

### Entrenando la red

El siguiente paso es entrenar la red. Para ello vamos a utilizar el [siguiente procedimiento](https://sgugger.github.io/the-1cycle-policy.html) basado en la idea de fine-tuning:

1. En primer lugar se dejan fijos (congelados) los pesos de la mayoría de capas de la red y sólo se actualizan los de las últimas capas. 
2. Se descongelan todas las capas de la red. 
3. Se reentrenan todas las capas de la red pero utilizando distintos *learning rates* en cada capa. 

La librería fastai proporciona toda la funcionalidad necesaria para llevar a cabo este proceso mediante el método `fine_tune`.

In [ ]:
learn.fine_tune(10,base_lr=1e-3)

Para su uso posterior, es conveniente exportar el modelo una vez entrenado. 

In [ ]:
learn.export()

Podemos ver que dicho modelo se ha guardado en el mismo directorio donde nos encontramos.

In [ ]:
Path().ls(file_exts='.pkl')

## Interpretación de los resultados

Vamos a interpretar los resultados utilizando la matriz de confusión.

A continuación se crear una interpretación de los resultados obtenidos con la misma. 

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

Por último mostramos la matriz de confusión obtenida. Además de la matriz de confusión se puede obtener [otra información](https://docs.fast.ai/vision.learner.html#ClassificationInterpretation).

In [ ]:
interp.plot_confusion_matrix()

Como podemos ver en la matriz de confusión anterior, el modelo tiende a confundir a Leela con Fry, esto puede deberse a que tengamos cierto ruido en nuestras imágenes (por ejemplo, imágenes que contengan a ambos personajes). Por lo tanto es conveniente limpiar nuestro dataset.

### Evaluando el modelo en el conjunto de test

Para poder evaluar nuestro modelo en el conjunto de test debemos crear un nuevo `DataBlock` y un nuevo `DataLoader`. La única diferencia con el `DataBlock` utilizado previamente es que para hacer la partición del dataset usamos un objeto de la clase `GrandparentSplitter` indicando que el conjunto de validación es nuestro conjunto de test. En el caso del `DataLoader`, la diferencia con el definido anteriormente es que cambiamos la ruta al path. 

In [ ]:
dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = Resize(256),
                 batch_tfms=aug_transforms(size=128,min_scale=0.75))
dlsTest = dbTest.dataloaders(path,bs=128)

Para trabajar con este dataloader debemos modificar nuestro objeto `Learner`. En concreto su atributo `dls`. 

In [ ]:
learn.dls = dlsTest

Ahora podemos evaluar nuestro modelo usando el método `validate`.

In [ ]:
learn.validate()

El método `validate` nos devuelve dos valores: el valor de la función de pérdida, y el valor de nuestra métrica (la accuracy en este caso). Por lo que podemos ver que el modelo tiene una accuracy en el conjunto de test de aproximadamente un 71% (esto puede variar dependiendo de la ejecución). 

## Limpiando el dataset

Como hemos comentado anteriormente puede ocurrir que haya imágenes en nuestro dataset que no deberían estar ahí.


In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner

En primer lugar podemos ver aquellas imágenes que tienen una mayor pérdida (es decir, aquellas que el modelo clasifica peor). Esto se puede hacer con ``.plot_top_losses``. 

In [ ]:
interp.plot_top_losses(5,nrows=1)

A continuación podemos utilizar el siguiente widget para limpiar el dataset.

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner
cleaner = ImageClassifierCleaner(learn)
cleaner

Una vez que hayamos seleccionados para eliminar, podemos usar el siguiente comando.

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

Como se puede apreciar hay ciertas imágenes en nuestro dataset que no son correctas, por lo que deberíamos hacer una limpieza del mismo para conseguir mejores resultados. 

## Poniendo el modelo en producción

Lo último que vamos a ver es cómo se puede poner el modelo en producción para usarlo para predecir la categoría de nuevas imágenes. 

Lo primero debemos exportar el modelo. La siguiente instrucción crea un fichero llamado 'export.pkl' en el directorio donde estamos trabajando (está almacenado en la variable ``path``) que sirve para desplegar el modelo. 

In [ ]:
learn.export()

Podemos ver que se ha creado dicho fichero.

In [ ]:
!ls data/futurama

A continuación indicamos al sistema que use la CPU para el proceso de inferncia (en caso de que el ordenador donde se despliega el modelo no tenga una GPU esto ocurre de manera automática).

In [ ]:
defaults.device = torch.device('cpu')

Vamos a probar nuestro modelo con una nueva imagen, en este caso de Fry. Comenzamos descargando dicha imagen, y a continuación la abrimos.


In [ ]:
!wget https://s.tcdn.co/802/3d8/8023d8dd-aada-3ff0-a91e-1c1b75b56a65/1.png -O fry.png

In [ ]:
import PIL
img = PILImage.create('fry.png')
img

A continuación creamos nuestro ``Learner``.

In [ ]:
learn_inf = load_learner('export.pkl')

Y por último realizamos la predicción. 

In [ ]:
pred_class,pred_idx,outputs=learn_inf.predict(img)
pred_class

La función `predict` devuelve tres valores:
- La clase (buildings en este caso).
- El índice asociado a dicha clase. 
- Las probabilidades para cada una de las categorías.  